In [15]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
import time

# ----------------------
# Black-Scholes Model
# ----------------------
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    elif option_type == 'put':
        return K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)

# ----------------------
# Binomial Tree Model
# ----------------------
def binomial_tree_price(S, K, T, r, sigma, steps=100, option_type='call'):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    q = (np.exp(r*dt) - d) / (u - d)

    prices = np.array([S * (u**j) * (d**(steps-j)) for j in range(steps+1)])
    if option_type == 'call':
        values = np.maximum(prices - K, 0)
    else:
        values = np.maximum(K - prices, 0)

    for i in range(steps-1, -1, -1):
        values = np.exp(-r*dt) * (q * values[1:] + (1-q) * values[:-1])
    return values[0]

# ----------------------
# Implied Volatility Solver
# ----------------------
def implied_volatility(S, K, T, r, market_price, option_type='call'):
    objective = lambda sigma: abs(black_scholes_price(S, K, T, r, sigma, option_type) - market_price)
    result = minimize_scalar(objective, bounds=(1e-5, 3), method='bounded')
    return result.x if result.success else None

# ----------------------
# Greeks Calculation
# ----------------------
def black_scholes_greeks(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        delta = norm.cdf(d1)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))) - r * K * np.exp(-r*T) * norm.cdf(d2)
        rho = K * T * np.exp(-r*T) * norm.cdf(d2)
    elif option_type == 'put':
        delta = norm.cdf(d1) - 1
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))) + r * K * np.exp(-r*T) * norm.cdf(-d2)
        rho = -K * T * np.exp(-r*T) * norm.cdf(-d2)

    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T)

    return {
        "Delta": delta,
        "Gamma": gamma,
        "Theta": theta,
        "Vega": vega,
        "Rho": rho
    }


# ----------------------
# Example Use Case: Alphabet (GOOGL)
# ----------------------
if __name__ == "__main__":
        S = 169
        today = datetime.today() - timedelta(days=1)
        expiry_date = datetime(today.year, 4, 25)
        T_call = (expiry_date - today).days / 365
        T_put = T_call
        r = 0.04  # 4% annual risk-free rate
        sigma = 0.3  # assumed volatility

        print("\n--- GOOGL Option Pricing Example ---")
        print(f"Spot Price (S): {S:.2f}, Time to Expiration: {T_call:.3f} years")

        # Call Option
        K_call = 200
        bs_call_price = black_scholes_price(S, K_call, T_call, r, sigma, 'call')
        bt_call_price = binomial_tree_price(S, K_call, T_call, r, sigma, 100, 'call')
        print(f"\nCall Option (K = 200):")
        print(f"Black-Scholes Price: {bs_call_price:.2f}")
        print(f"Binomial Tree Price: {bt_call_price:.2f}")

        # Put Option
        K_put = 190
        bs_put_price = black_scholes_price(S, K_put, T_put, r, sigma, 'put')
        bt_put_price = binomial_tree_price(S, K_put, T_put, r, sigma, 100, 'put')
        print(f"\nPut Option (K = 190):")
        print(f"Black-Scholes Price: {bs_put_price:.2f}")
        print(f"Binomial Tree Price: {bt_put_price:.2f}")

        market_price_call = 3.50  # hypothetical market price for the call option

        # Calculate Implied Volatility
        K=200
        T=0.082
        iv_call = implied_volatility(S, K, T, r, market_price_call, 'call')

        # Calculate Greeks using implied volatility just calculated
        greeks_call = black_scholes_greeks(S, K, T, r, iv_call)

        print(f"\nImplied Volatility (Call): {iv_call:.2f}")
        print("\nGreeks (Call):")
        for greek, value in greeks_call.items():
            print(f"{greek}: {value:.2f}")

        # Hypothetical market price for the put option
        market_price_put = 25.00

        # Implied volatility calculation for put
        K_put = 190
        iv_put = implied_volatility(S, K_put, T_put, r, market_price_put, 'put')

        # Greeks calculation for put option using calculated IV
        greeks_put = black_scholes_greeks(S, K_put, T_put, r, iv_put, 'put')

        # Output the results
        print(f"\nImplied Volatility (Put): {iv_put:.2f}")
        print("\nGreeks (Put):")
        for greek, value in greeks_put.items():
            print(f"{greek}: {value:.2f}")


--- GOOGL Option Pricing Example ---
Spot Price (S): 169.00, Time to Expiration: 0.079 years

Call Option (K = 200):
Black-Scholes Price: 0.15
Binomial Tree Price: 0.14

Put Option (K = 190):
Black-Scholes Price: 21.02
Binomial Tree Price: 21.01

Implied Volatility (Call): 0.65

Greeks (Call):
Delta: 0.21
Gamma: 0.01
Theta: -57.09
Vega: 14.08
Rho: 2.67

Implied Volatility (Put): 0.61

Greeks (Put):
Delta: -0.72
Gamma: 0.01
Theta: -55.30
Vega: 16.04
Rho: -11.65


Call Option (Strike = $200) assessment results

Option Price:
    Black-Scholes: $0.15
    Binomial Tree: $0.14
Implied Volatility (IV): 65% (high, suggesting the market expects significant price moves)
Greeks Summary:
    Delta (0.21): Relatively low sensitivity to underlying price (out-of-the-money).
    Gamma (0.01): Small sensitivity of Delta to price changes.
    Theta (-57.09): Very high daily time decay, significantly reducing option value each day.
    Vega (14.08): Highly sensitive to volatility changes (benefits from volatility spikes).
    Rho (2.67): Minor positive impact if interest rates rise.
Insight: The call is inexpensive, indicating it's deep out-of-the-money and risky for short-term speculation. High Theta implies rapid value erosion.

Put Option (Strike = $190) assessment results

Option Price:
    Black-Scholes: $21.02
    Binomial Tree: $21.01
Implied Volatility (IV): 61% (also relatively high)
Greeks Summary:
    Delta (-0.72): High sensitivity, significant gains if the underlying asset falls.
    Gamma (0.01): Small incremental Delta changes with price moves.
    Theta (-55.30): Substantial daily value erosion, but slightly lower than call.
    Vega (16.04): Even higher sensitivity to volatility than the call; beneficial in volatile markets.
    Rho (-11.65): Negative sensitivity; loses value if interest rates rise.
Insight: The put option is in-the-money, expensive, and strongly positioned to profit if GOOGL price falls soon. However, significant daily time decay suggests caution in holding too long without price moves.